In [ ]:
!pip install -U -q transformers datasets bitsandbytes trl peft

ERROR: Could not find a version that satisfies the requirement transfomers (from versions: none)
ERROR: No matching distribution found for transfomers


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-v0.3"

config_4bit = BitsAndBytesConfig(load_in_4bit=True)

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=config_4bit,
    device_map="auto",
    trust_remote_code=True
)




Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left", trust_remote_code=True)

In [7]:
from datasets import load_dataset

dataset = load_dataset("MattCoddity/dockerNLcommands")
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 2415
    })
})

In [8]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>']

In [9]:
tokens =['<system>',"<user>","<assistant>"]

for token in tokens:
    token_id = tokenizer.encode(token,add_special_tokens=False)
    print(token, token_id)

<system> [1291, 7342, 29535]
<user> [1291, 2606, 29535]
<assistant> [1291, 1257, 11911, 29535]


In [10]:
tokenizer.pad_token

In [11]:
special_token = {
    'pad_token': '<PAD>',
    'additional_special_tokens': ['<system>', '<user>', '<assistant>']
}
tokenizer.add_special_tokens(special_token)

4

In [12]:
tokens = ['<system>', '<user>', '<assistant>', '<PAD>']
for token in tokens:
    token_id = tokenizer.encode(token, add_special_tokens=False)
    print(token, token_id)

<system> [32769]
<user> [32770]
<assistant> [32771]
<PAD> [32768]


In [13]:
len(tokenizer)

32772

In [14]:
model_4bit.config.vocab_size

32768

In [15]:
model_4bit.resize_token_embeddings(len(tokenizer))

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(32772, 4096)

In [17]:
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '<PAD>', '<system>', '<user>', '<assistant>']

In [18]:
tokenizer.bos_token

'<s>'

In [19]:
tokenizer.eos_token

'</s>'

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction'],
        num_rows: 2415
    })
})

In [22]:
new_template = """<s><system>{system_prompt}</s><user>{user_prompt}</s><assistant>{model_answer}</s>"""

def format_dataset(example):
    system_prompt = example["instruction"]
    user_prompt = example["input"]
    model_answer = example["output"]

    formatted_text = new_template.format(
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        model_answer=model_answer
    )

    return {"text": formatted_text}

In [23]:
format_dataset(dataset['train'][0])

{'text': "<s><system>translate this sentence in docker command</s><user>Give me a list of containers that have the Ubuntu image as their ancestor.</s><assistant>docker ps --filter 'ancestor=ubuntu'</s>"}

In [24]:
dataset = dataset.map(format_dataset)


Map:   0%|          | 0/2415 [00:00<?, ? examples/s]

In [25]:
dataset['train']['text'][0]

"<s><system>translate this sentence in docker command</s><user>Give me a list of containers that have the Ubuntu image as their ancestor.</s><assistant>docker ps --filter 'ancestor=ubuntu'</s>"